In [1]:
#Importing Libraries
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

import os
from itertools import repeat

In [2]:
data_loc = "C:/Users/Charan.Teja/Charan new onedrive/OneDrive - TEG Business Solutions Pvt Ltd/work data/AEP Work/AEP_2025/Mock_Run/Data_Preparation/SSBCI/Intermediate/"

In [3]:
def add_trailing_zeros(s, num_zeros):
    # repeat the zero character and add it to the end of the string
    padded_string = s + "".join(repeat("0", num_zeros))
    return padded_string

In [4]:
data = pd.read_csv(data_loc + "add_ben_vbid_19_24_input.csv")
data.head(2)

,bid_id,pbp_vbid_group_id,pbp_b19b_states_ehc_vbd,Year
0,H0028_007_0,1,0.0,2024
1,H0028_015_0,1,0.0,2024


### No. of pos

In [5]:
pos_no = 11

In [6]:
pos_list = []
for i in range(pos_no):
    pos_list.append("pos"+"_"+str(i+1))

In [7]:
data[pos_list] = np.nan

data['pbp_b19b_states_ehc_vbd'].fillna(0,inplace=True)

In [8]:
data['pbp_b19b_states_ehc_vbd'].value_counts()

pbp_b19b_states_ehc_vbd
0.000000e+00    987
1.111111e+09     49
1.100010e+09     12
1.000000e+08      9
1.000000e+06      8
1.000000e+09      2
Name: count, dtype: int64

In [9]:
count = 0
for i in range(data.shape[0]):
    if len(str(int(data['pbp_b19b_states_ehc_vbd'].iloc[i]))) != 11:
        data['pbp_b19b_states_ehc_vbd'].iloc[i] = str(int(data['pbp_b19b_states_ehc_vbd'].iloc[i])).zfill(10)
    else:
        count = count + 1
        #print(int(data['pbp_b19b_states_ehc_vbd'].iloc[i]))
        data['pbp_b19b_states_ehc_vbd'].iloc[i] = str(int(data['pbp_b19b_states_ehc_vbd'].iloc[i]))

In [10]:
data['pbp_b19b_states_ehc_vbd'].value_counts()

pbp_b19b_states_ehc_vbd
0000000000    987
1111111111     49
1100010001     12
0100000000      9
0001000000      8
1000000000      2
Name: count, dtype: int64

In [11]:
for i in range(data.shape[0]):
    if len(str(int(data['pbp_b19b_states_ehc_vbd'].iloc[i]))) != 11:
        data['pbp_b19b_states_ehc_vbd'].iloc[i] = add_trailing_zeros(data['pbp_b19b_states_ehc_vbd'].iloc[i],1)

In [12]:
data['pbp_b19b_states_ehc_vbd'].value_counts()

pbp_b19b_states_ehc_vbd
00000000000    987
11111111110     49
11000100010     12
01000000000      9
00010000000      8
10000000000      2
Name: count, dtype: int64

In [13]:
for i in range(data.shape[0]):
    for j in range(11):
        if data['pbp_b19b_states_ehc_vbd'].iloc[i][j] == '0':
            data['pos_'+str(j+1)].iloc[i] = 0
        elif data['pbp_b19b_states_ehc_vbd'].iloc[i][j] == '1':
            data['pos_'+str(j+1)].iloc[i] = 1

### Bringing prevalance data

In [14]:
# v19_loc = "C:/Users/Charan.Teja/Charan new onedrive/TEG Business Solutions Pvt Ltd/HWAI - TEG-HealthWorks/Data Science/xAI 2023_Feb/Product/4.Modelling/Files/Final_data_Rerun/"
loc_24 = "C:/Users/Charan.Teja/Charan new onedrive/TEG Business Solutions Pvt Ltd/HWAI - TEG-HealthWorks/AEP Refresh/AEP 2025/AEP 2025 - Dry Run/3. Output Data/Landscape/"

In [15]:
# lscape_19_23 = pd.read_csv(v19_loc + 'Final_data_v19.csv')
# lscape_19_23 = lscape_19_23[['State','County','bid_id','Year']]

lscape_24 = pd.read_csv(loc_24 + 'Landscape_dry_run_10thSept.csv')
lscape_24 = lscape_24[['State','County','key_segment1']]
lscape_24.rename(columns={'key_segment1':'bid_id'}, inplace=True)
lscape_24['Year'] = 2024

# lscape_19_24 = pd.concat([lscape_19_23, lscape_24])
lscape_19_24 = lscape_24.copy()

In [16]:
data_lscape_19_24 = lscape_19_24.merge(data, on=['bid_id','Year'], how='left')

data_lscape_19_24.fillna(0, inplace=True)

In [17]:
mapping_df = pd.read_excel('Chronic cond mapping.xlsx', sheet_name='main data')
vbid_map_df = pd.read_excel('Chronic cond mapping.xlsx', sheet_name='vbid')
ssbci_map_df = pd.read_excel('Chronic cond mapping.xlsx', sheet_name='ssbci')
uf_map_df = pd.read_excel('Chronic cond mapping.xlsx', sheet_name='uf')

CC_Prevalance_data_cleaned=pd.read_csv("CC_Prevalance_data_cleaned.csv")

In [18]:
merged=pd.merge(vbid_map_df,mapping_df,on='Index',how='left')

data_lscape_19_24_melted = pd.melt(data_lscape_19_24, id_vars=['State', 'County', 'bid_id', 'Year','pbp_vbid_group_id','pbp_b19b_states_ehc_vbd'], value_vars=['pos_1', 'pos_2', 'pos_3', 'pos_4', 'pos_5', 'pos_6', 'pos_7', 'pos_8',
       'pos_9', 'pos_10', 'pos_11'])

In [19]:
data_lscape_19_24_melted.shape

(1582614, 8)

In [20]:
merged_data = pd.merge(data_lscape_19_24_melted, merged, left_on='variable', right_on='pos_add_ben')

In [21]:
CC_Prevalance_data_cleaned_melted= pd.melt(CC_Prevalance_data_cleaned, id_vars=['State', 'County'])
CC_Prevalance_data_cleaned_melted.rename(columns={'variable':'CMS data', 'value':'value2'},inplace=True)
final_result = pd.merge(merged_data, CC_Prevalance_data_cleaned_melted, on=['State', 'County','CMS data'], how='left')

In [22]:
final_result['final_values'] = final_result['value'].astype(float) * final_result['value2'].astype(float)
final_result=final_result.pivot_table(values=['final_values'], index=['State', 'County', 'bid_id', 'Year','pbp_vbid_group_id','pbp_b19b_states_ehc_vbd'], columns='variable', aggfunc='sum')

In [23]:
final_result.columns =[s2  for (s1,s2) in final_result.columns.tolist()]
final_result.reset_index(inplace=True)

In [24]:
data=final_result.copy()

In [25]:
data['No_of_chronic_states_vbid'] = data[pos_list].sum(axis=1)
data=data.groupby(by=['State','County','bid_id','Year'],as_index=False).agg('sum')

In [26]:
op_loc = 'C:/Users/Charan.Teja/Charan new onedrive/OneDrive - TEG Business Solutions Pvt Ltd/work data/AEP Work/AEP_2025/Mock_Run/Data_Preparation/SSBCI/output/'
data.to_csv(op_loc+"Add_ben_chronic_cond_vbid_final.csv",index=False)